# Introduction to numerical robotics

This notebook is a general introduction to Pinocchio. It shows how to manipulate the geometry model of a robot manipulator: set the configuration, compute the position of the end effector, check for collisions or the distance to an obstacle. The main idea is to give a brief introduction of the general topic: how to discover and learn a robot movement constrained by the environment, using iterative optimization methods.


In [15]:
import magic_donotload

## Set up

Let us load the UR5 robot model, the Pinocchio library, some optimization functions from SciPy and the Matplotlib for plotting:

In [16]:
import pinocchio as pin
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
import time
import numpy as np
from numpy.linalg import inv,norm,pinv,svd,eig
from scipy.optimize import fmin_bfgs,fmin_slsqp
from utils.load_ur5_with_obstacles import load_ur5_with_obstacles,Target
import matplotlib.pylab as plt

Let's first load the robot model and display it. For this tutorial, a single utility function will load the robot model and create obstacles around it:

In [17]:
robot = load_ur5_with_obstacles(reduced=True)

The next few lines initialize a 3D viewer.

In [18]:
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [19]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

The robot and the red obstacles are encoded in the `robot` object (we will not look in depth at what is inside this object). You can display a new configuration of the robot with `viz.display`. It takes a `numpy.array` of dimension 2 as input:

In [20]:
viz.display(np.array([3.,-1.5])) 

We also set up a target with is visualized as a green dot:

In [21]:
target_pos = np.array([.5,.5])
target = Target(viz,position = target_pos)

The `Target` object is the green dot that the robot should reach. You can change the target position by editing `target.position`, and display the new position with `target.display()`.

## Using the robot model
The robot is originally a 6 degrees-of-freedom (DOF) manipulator. Yet to make the example simple, we will only use its joints 1 and 2. The model has simply be loaded with "frozen" extra joints, which will then not appear in this notebook. Reload the model with `reduced=False` if you want to recover a model with full DOF.

The following function computes the position of the end effector (in 2d):

In [22]:
def get_position_end_eff(q):
     '''
     Return the 2d position of the end effector.
     '''
     pin.framesForwardKinematics(robot.model,robot.data,q)
     return robot.data.oMf[-1].translation[[0,2]]

This function checks if the robot is in collision, and returns `True` if a collision is detected.

In [23]:
def check_collision(q):
     '''
     Return true if in collision, false otherwise.
     '''
     pin.updateGeometryPlacements(robot.model,robot.data,robot.collision_model,robot.collision_data,q)
     return pin.computeCollisions(robot.collision_model,robot.collision_data,False)

The next function computes the distance between the end effector and the target.

Your code:

In [24]:
def compute_distance_end_target(q):
     '''
     Return the distance between the end effector end the target (2d).
     '''
     return norm(get_position_end_eff(q)-target.position)

Solution

In [ ]:
%do_not_load tp0/generated/simple_path_planning_dist

## Random search of a valid configuration
The free space is difficult to represent explicitely. We can sample the configuration space until a free configuration is found:

In [25]:
def sample_random_config(check=False):
    '''
    Return a random configuration. If check is True, this
    configuration is not is collision
    '''
    while True:
        q = np.random.rand(2)*6.4-3.2  # sample between -3.2 and +3.2.
        if not check or not check_collision(q):
            return q

The solution if needed:

In [ ]:
%do_not_load tp0/generated/simple_path_planning_qrand

In [26]:
viz.display(sample_random_config(check=True))

Let's now find a valid configuration that is arbitrarily close to the target: sample until dist is small enough and coll is false (you may want to display the random trials inside the loop).

## From a random configuration to the target
Let' s now start from a random configuration. How can we find a path that bring the robot toward the target without touching the obstacles. Any idea?

In [28]:
def walk_random_descent(q0 = None):
     
     q = sample_random_config(check=True) if q0 is None else q0
     hist = [ q.copy() ]
     
     for i in range(100):
          
          # Choose a random step and apply 
          dq = sample_random_config()*.1 
          qtry = q + dq 
                    
          # if distance decreases without collision ... 
          if compute_distance_end_target(q) > compute_distance_end_target(qtry) and not check_collision(qtry): 
               
               q = qtry                 # keep the step
               hist.append(q.copy())    # keep a trace of it
               viz.display(q)           # display it
               time.sleep(5e-3)         # and sleep for a short while
               
     return hist 

Clue:
- Use a random walk sampling random direction to extend
- Bias the walk toward the goal by keeping the extension only if it is closer to the goal

And solution if needed

In [ ]:
%do_not_load tp0/generated/simple_path_planning_random_descent

In [29]:
walk_random_descent()

[array([-2.56381286, -0.72086863]),
 array([-2.4239068 , -0.99442995]),
 array([-2.1314995, -1.0772491]),
 array([-1.87030078, -1.27519837]),
 array([-1.77790827, -1.19561702]),
 array([-1.62844343, -1.1323628 ]),
 array([-1.61475491, -1.18601852]),
 array([-1.55420437, -1.32294711]),
 array([-1.50495762, -1.42575317]),
 array([-1.43034855, -1.45791769]),
 array([-1.36126248, -1.75964241]),
 array([-1.18489876, -1.86621032]),
 array([-1.08627642, -2.09805074]),
 array([-1.04020838, -2.37305361]),
 array([-0.87445965, -2.2544713 ])]

## Configuration space
Let's try to have a better look of the configuration space. In this case, it is easy, as it is dimension 2: we can sample it exhaustively and plot it in 2D. For that, let's introduce another function to compute the distance to collision:

In [ ]:
def compute_min_distance_env(q):
     ''' 
     Return the minimal distance between robot and environment. 
     ''' 
     
     pin.updateGeometryPlacements(robot.model,robot.data,robot.collision_model,robot.collision_data,q)
     
     is_collision = pin.computeCollisions(robot.collision_model,robot.collision_data,False)
     
     if is_collision: 
         return 0.0 
    
     idx = pin.computeDistances(robot.collision_model,robot.collision_data)
     min_distance = robot.collision_data.distanceResults[idx].min_distance 
     
     return min_distance 

Now, let's sample the configuration space and plot the distance-to-target and the distance-to-obstacle field (I put 500 samples to spare your CPU, but you need at least 10x more for obtaining a good picture).

In [ ]:
def sample_config_space(num_samples = 500):
     ''' 
     Sample num_samples configurations and store them in two lists 
     depending if the configuration is in free space (hfree) or in 
     collision (hcol), along with the distance to the target and the 
     distance to the obstacles. 
     ''' 
     
     hist_coll = []
     hist_free = []
     
     for i in range(num_samples):
          
          q = sample_random_config(False)
          
          if not check_collision(q):
               hist_free.append( list(q.flat) + [ compute_distance_end_target(q), find_min_collision_distance(q) ])
          else:
               hist_coll.append( list(q.flat) + [ compute_distance_end_target(q), 1e-2 ]) 
               
     return hist_coll, hist_free 

# ---------------------------------- 

def plot_config_space(hist_coll, hist_free, marker_size = 20):
     '''
     Plot 2 "scatter" plots: the first one plot the distance to the target for 
     each configuration, the second plots the distance to the obstacles (axis q1,q2, 
     distance in the color space).
     '''
     
     hist_total = hist_coll + hist_free 
     h = np.array(hist_total) 
     
     plt.subplot(2,1,1)
     plt.scatter(h[:,0], h[:,1], c = h[:,2], s = marker_size,lw = 0)
     plt.title("Distance to the target")
     plt.colorbar() 
     
     plt.subplot(2,1,2)
     plt.scatter(h[:,0], h[:,1], c = h[:,3], s = marker_size, lw = 0)
     plt.title("Distance to the obstacles")
     plt.colorbar() 

In [ ]:
hist_coll, hist_free = sample_config_space(5000) 

You can try to match your representation of the free space of the robot with this plot. 
As an example, you can display on this plot a feasible trajectory discover by random walk from an init position.

In [ ]:
# Plot random trajectories in the same plot
qinit = np.array([-1.1, -3. ])
for i in range(100):
     traj = walk_random_descent(qinit)
     if compute_distance_end_target(traj[-1])<5e-2:
          print('We found a good traj!')
          break
traj = np.array(traj)

# Chose trajectory end to be in [-pi,pi]
qend = (traj[-1]+np.pi) % (2*np.pi) - np.pi

# Take the entire trajectory it modulo 2 pi
traj += (qend-traj[-1])

plt.plot(traj[:,0],traj[:,1],'r',lw=5) 

Here is a solution:

In [ ]:
%do_not_load tp0/generated/simple_path_planning_traj

In [ ]:
# Add your traj to the plot, be careful !

plot_config_space(hist_coll, hist_free) 

plt.plot(traj[:,0],traj[:,1],'r',lw=5) 

## Optimize the distance under non-collision constraint
Finally, let's use one of the optimizers from SciPy to search for a robot configuration that minimizes the distance to the target, under the constraint that the distance to collision is positive.
For that, we define a *cost function* $cost: \mathcal{C} \to \mathbb{R}$ (taking the robot configuration and returning a scalar) and a constraint function (taking again the robot configuration and returning a scalar that should be positive). We additionally use the "callback" functionnality of the solver to render the robot configuration corresponding to the current value of the decision variable inside the solver algorithm.
We use the "SLSQP" solver from SciPy, which implements a "sequential quadratic program" algorithm and accepts constraints.


The constraint is a positive distance with a margin on $\epsilon$

In [ ]:
epsilon = .01

In [ ]:
def compute_cost(q):
     '''
     Cost function: distance to the target.
     '''
     return compute_distance_end_target(q) ** 2


def define_constraint(q):
     '''
     Constraint function: distance to the obstacle should be strictly positive.
     '''
     min_collision_dist = 0.01  # [m]
     return find_min_collision_distance(q) - min_collision_dist


def display_callback(q):
     ''' 
     At each optimization step, display the robot configuration.
     ''' 
     viz.display(q)
     time.sleep(0.01)


def optimize():
     '''
     Optimize from an initial random configuration to discover a 
     collision-free configuration as close as possible to the target. 

     Returns: results from fmin_slsqp optimization tuple containing:
          - x:     The optimized configuration
          - fx:    Final function value 
          - its:   Number of iterations
          - imode: Exit mode from optimizer
          - smode: Status message
     '''
     
     # Start from a random configuration that is collision-free
     x0 = sample_random_config(check = True)

     # Set objective function (minimize distance to target)
     function = compute_cost

     # Set constraint function (stay away from obstacles)
     inequality_constraints = define_constraint

     # Function to call at each iteration to visualize progress
     callback = display_callback

     # Get detailed output from optimizer
     full_output = 1
     
     # Run Sequential Least Squares Programming optimization
     return fmin_slsqp(  x0          = x0,
                         func        = function,
                         f_ieqcons   = inequality_constraints,
                         callback    = callback,
                         full_output = full_output)
     
optimize() 

Here is a valid solution:

In [ ]:
%do_not_load tp0/generated/simple_path_planning_optim

Look at the output of the solver. It always returns a variable value, but sometimes the algorithm fails being traped in an unfeasible region. Most of the time, the solver converges to a local minimum where the final distance to the target is nonzero

Now you can write a planner that try to optimize and retry until a valid solition is found!

In [ ]:
# Your solution
while True:
     
    result = optimize()
    q      = result[0]
    
    viz.display(q)
    
    if result[4] == 'Optimization terminated successfully' and result[1]<1e-6:
        print('Finally successful!')
        break 
   
    print("Failed ... let's try again! ")

And the solution if you need it:

In [ ]:
%do_not_load tp0/generated/simple_path_planning_useit